In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.optimize as sco
import matplotlib.pyplot as plt
import random

In [ ]:
alpha_df.set_index("code",inplace=True)

#去除st股票
st_stock=list(pd.read_csv("st_stocks.csv")["code"])
alpha_df.drop(st_stock,inplace=True)

#去除新股和停牌股
grouped=alpha_df.groupby("datetime")
date_list=list(alpha_df["datetime"].drop_duplicates())
code_list=list(alpha_df.index.drop_duplicates())
join=set(code_list)
union=set(code_list)
for date in date_list:
    df=grouped.get_group(date)
    stock_set=set(df.index)
    join=stock_set&join
    union=stock_set|union
other_list=list(union-join)
alpha_df.drop(other_list,inplace=True)

alpha_df.to_csv("growth_factor.csv",index_label="code")

In [ ]:
#数据处理
change_data_1=pd.read_csv("change.csv",dtype={"stock_code":str})
change_data_2=pd.read_csv("change_2.csv",dtype={"stock_code":str})
change_data=pd.concat([change_data_1,change_data_2])

change_data.rename(columns={"stock_code":"code","trade_date":"datetime"},inplace=True)
change_data["datetime"]=change_data["datetime"].apply(lambda x:x[0:10])
change_data["code"]=change_data["code"].apply(lambda x:x+".SH" if x[0]=="6" else x+".SZ")

change_data.sort_values(by=["datetime","code"],ascending=[True,True],inplace=True)

change_data.set_index("code",inplace=True)

grouped=change_data.groupby("datetime")
date_list=list(change_data["datetime"].drop_duplicates())
code_list=list(change_data.index.drop_duplicates())
join=set(code_list)
union=set(code_list)
for date in date_list:
    df=grouped.get_group(date)
    stock_set=set(df.index)
    join=stock_set&join
    union=stock_set|union
other_list=list(union-join)
change_data.drop(other_list,inplace=True)

drop_list=random.sample(list(join),1135)
change_data.drop(drop_list,inplace=True)

change_data.reset_index(inplace=True)


data=pd.read_csv("data_needs.csv")
date=list(data["datetime"].drop_duplicates())[-1]
df=data.groupby("datetime").get_group(date)
dic=dict(zip(df["code"],df["industry_code"]))
change_data["industry_code"]=change_data["code"].apply(lambda x:dic.get(x,0))

drop_list=list(set(change_data["code"][change_data["industry_code"]==0]))
change_data.set_index("code",inplace=True)
change_data.drop(drop_list,inplace=True)
change_data.reset_index(inplace=True)

stock_ret_df=change_data.pivot_table("change_rate",index="datetime",columns="code")

In [ ]:
#读入各行业日频收益率数据并进行形式上的处理
data_df=pd.read_csv("indus_ret.csv")
data_df.sort_values(by=["trade_date","industry_code"],ascending=["True","True"],inplace=True)

data_df.reset_index(drop=True,inplace=True)

indus_ret_df=data_df.pivot_table("change_rate",index="trade_date",columns="industry_code")

In [ ]:
def fun(index,stock_ret,indus_ret_df):
    
    half_time=60
    lamd=pow(0.5,1/half_time)
    weight=np.power(lamd,np.arange(252))[::-1]
    weight=weight/np.sum(weight)
    
    indus_ret_df_1=pd.DataFrame(zjh(np.array(indus_ret_df.iloc[index,:])))
    
    beta_ls=[]
    for indus in indus_ret_df_1.columns:
        x=np.array(indus_ret_df_1[indus])
        y=np.array(stock_ret.iloc[index])
        x=sm.add_constant(x)
        y=np.dot(np.diag(weight),y)
        x=np.dot(np.diag(weight),x)
        try:
            model=sm.OLS(y,x)
            re=model.fit()
            beta_ls.append(re.params[1])
        except:
            return np.nan
    
    beta_arr=np.array(beta_ls)

    beta_arr=np.abs(beta_arr)/np.sum(np.abs(beta_arr))
    
    beta_df.loc[(stock_ret.index[int(index[-1])],stock_code)]=beta_arr
    
    return 0

In [ ]:
def zjh(arr):
    arr_t=arr.transpose()

    #计算重叠矩阵
    overlap_matrix=np.dot(arr_t,arr)
    overlap_matrix=(overlap_matrix.T+overlap_matrix)/2
    #计算特征值、特征向量
    eigvalue,eigvector=np.linalg.eig(overlap_matrix)

    #处理对角矩阵
    eigvalue=eigvalue**(-0.5)
    DiagMat=np.diag(eigvalue)

    #计算最终正交矩阵
    tmp=np.dot(eigvector,DiagMat)
    TSMat=np.dot(tmp,eigvector.transpose())
    OrthMat=np.dot(arr,TSMat)
    
    return OrthMat

In [ ]:
beta_df=pd.DataFrame(columns=indus_ret_df.columns,index=pd.MultiIndex.from_product([stock_ret_df.index[251::21],stock_ret_df.columns]))

In [ ]:
date_num=stock_ret_df.index
index_se=pd.Series(np.arange(len(date_num)))

for stock_code in stock_ret_df.columns:
    print(stock_code)
    stock_ret=stock_ret_df[stock_code]
    
    for i in range(251,len(date_num),21):
        fun(range(i-251,i+1),stock_ret,indus_ret_df)

In [ ]:
'''
indus_dict=dict(zip(np.arange(28),indus_ret_df.columns))
beta_df["max_indus"]=beta_df.iloc[:,0:28].apply(lambda x:np.argmax(np.array(x)),axis=1)
beta_df["max_indus"]=beta_df["max_indus"].map(indus_dict)

beta_df.reset_index(inplace=True)
beta_df["real_indus"]=beta_df["level_1"].map(dic)
'''

In [ ]:
beta_df.reset_index(inplace=True)
beta_df.rename(columns={"level_0":"datetime","level_1":"code"},inplace=True)

In [ ]:
alpha_df=pd.read_csv("equal_weight ZScore_1.csv")

code_list=list(alpha_df["code"].drop_duplicates())
alpha_df.set_index("code",inplace=True)

drop_code=set(code_list)-set(stock_ret_df.columns)
alpha_df.drop(drop_code,inplace=True)

alpha_df.reset_index()

alpha_df.sort_values(by=["datetime","code"],ascending=[True,True],inplace=True)

alpha_df=pd.merge(alpha_df,beta_df,on=["datetime","code"])
alpha_df.fillna(0,inplace=True)

In [ ]:
mcap_data_1=pd.read_csv("mcap.csv",dtype={"stock_code":str})
mcap_data_2=pd.read_csv("mcap_2.csv",dtype={"stock_code":str})
mcap_data=pd.concat([mcap_data_1,mcap_data_2])

mcap_data.rename(columns={"stock_code":"code","trade_date":"datetime"},inplace=True)
mcap_data["datetime"]=mcap_data["datetime"].apply(lambda x:x[0:10])
mcap_data["code"]=mcap_data["code"].apply(lambda x:x+".SH" if x[0]=="6" else x+".SZ")

new_alpha_df=pd.merge(alpha_df,mcap_data,on=["datetime","code"])

In [ ]:
ret_df=stock_ret_df

for code in ret_df.columns:
    ret_df[code+"_In_change"]=np.array(np.log(1+ret_df[code]/100))[::-1]
    ret_df[code+"_21d_ret"]=(np.array(np.exp(ret_df[code+"_In_change"].rolling(21).apply(np.sum)))[::-1]-1)*100
    ret_df[code+"_21d_ret"]=ret_df[code+"_21d_ret"].shift(-1)
    del ret_df[code+"_In_change"]

stock_num=len(stock_ret_df.columns)
df=pd.DataFrame(stock_ret_df.iloc[:,int(stock_num/2):stock_num]).stack()

df=df.reset_index()
df["code"]=df["code"].apply(lambda x:x[0:9])
new_alpha_df=pd.merge(new_alpha_df,df,on=["datetime","code"])
new_alpha_df.rename(columns={0:"21d_ret"},inplace=True)

In [ ]:
zz500=pd.read_csv("zz500.csv")
zz500["_In_change"]=np.array(np.log(1+zz500["change_rate"]/100))[::-1]
zz500["_21d_ret"]=(np.array(np.exp(zz500["_In_change"].rolling(21).apply(np.sum)))[::-1]-1)*100
zz500["_21d_ret"]=zz500["_21d_ret"].shift(-1)


zz500.rename(columns={"_21d_ret":"mkt_ret","trade_date":"datetime"},inplace=True)
zz500["datetime"]=zz500["datetime"].apply(lambda x:x[0:10])
del zz500["index_code"]
del zz500["_In_change"]
del zz500["change_rate"]
new_alpha_df=pd.merge(new_alpha_df,zz500,on="datetime")

In [ ]:
def data_handle(mat):
    for i in range(mat.shape[1]):
        arr = np.array(mat[:,i])
        M = np.median(arr)
        MAD = 1.483 * np.median(np.abs(arr - M))
        arr[arr > M + 3 * MAD] = M + 3 * MAD
        arr[arr < M - 3 * MAD] = M - 3 * MAD
        mat[:,i] = arr
            
    for i in range(mat.shape[1]):
        arr = np.array(mat[:,i])
        weight = np.array(len(arr) * [1 / len(arr)])
        weighted_avarage = np.dot(weight.T, arr)
        arr = (arr - weighted_avarage) / arr.std()
        mat[:,i] = arr
    return mat

In [ ]:
raw_alpha_df=pd.read_csv("indus_factor_comp_1.csv")

In [ ]:
alpha_df=raw_alpha_df
alpha_df.set_index("datetime",inplace=True)

grouped=alpha_df.groupby("datetime")
factor_list=['BETA','RSTR','LNCAP','ETOP','DASTD','EGRO','BTOP','DTOA','STOM']
alpha_df["R_square_1"]=np.nan
alpha_df["R_square_2"]=np.nan

for date in date_list[251:-30:42]:
    df=grouped.get_group(date)

    mktv=np.array(df["mcap"])
    mktv_pct=np.sqrt(mktv)/np.sum(np.sqrt(mktv))
    weight=mktv_pct**(0.5)
    
    y=np.array(df["21d_ret"]-df["mkt_ret"])
    y=np.dot(np.diag(weight),y)
    
    #行业哑变量回归
    indus_factor=np.array(sm.categorical(np.array(df["industry_code"])))[:, 1:]
    style_factor=np.array(df[factor_list])
    style_factor=data_handle(zjh(style_factor))
    x=np.column_stack([indus_factor,style_factor])
    x=np.dot(np.diag(weight),x)
    
    model=sm.OLS(y,x)
    result=model.fit()
    R_square_1=result.rsquared_adj
    alpha_df["R_square_1"][date]=R_square_1
    
    #行业连续变量回归
    indus_factor=np.array(df[[str(x) for x in indus_ret_df.columns]])
    style_factor=np.array(df[factor_list])
    style_factor=data_handle(zjh(style_factor))
    x=np.column_stack([indus_factor,style_factor])
    x=np.dot(np.diag(weight),x)
    
    model=sm.OLS(y,x)
    result=model.fit()
    R_square_2=result.rsquared_adj
    alpha_df["R_square_2"][date]=R_square_2
    
alpha_df.reset_index(inplace=True) 
    
    

In [ ]:
df_1=alpha_df.groupby("datetime").agg(np.mean)[["R_square_1","R_square_2"]]
df_1.rename(columns={"R_square_1":"dummy_indus_vari","R_square_2":"beta_indus_vari"},inplace=True)

ax = df_1.plot() 
fig = ax.get_figure()
fig.savefig('fig_1.jpg',dpi=500)

df_1

In [ ]:
df_2=alpha_df.groupby("datetime").agg(np.mean)[["R_square_1","R_square_2"]]
df_2.rename(columns={"R_square_1":"dummy_indus_vari","R_square_2":"beta_indus_vari_half_time"},inplace=True)
df_2.dropna(inplace=True)


ax = df_2.plot() 
fig = ax.get_figure()
fig.savefig('fig_2.jpg',dpi=500)

df_2

In [ ]:
df=pd.concat([df_1,df_2],axis=1)
ax = df.plot() 
fig = ax.get_figure()
fig.savefig('fig_3.jpg',dpi=500)